In [ ]:
stopwords_list = ['stopwords.txt', 'stopwords_agri.txt', 'stopwords_mobi.txt']
from nltk.corpus import stopwords


class RefEnv(object):
	"""edit docs"""
	def __init__(self, path_PROJECT_DATA= '/Volumes/GoogleDrive/My Drive'):
		self.dst = 'Source'
		self.Export_folder = 'Export'
		self.path_PROJECT_DATA = path_PROJECT_DATA
		self.transformed = '{}/Open Data Platform/DATA/Transformed/{}'.format(self.path_PROJECT_DATA, '{}')




class Lemm(object): # returns a dataframe with new columns that are lemmatized with a suffix lemm but the data frame must contain the headers that are defined in header_tr 
    
    def __init__(self, file ='stopwords.txt'):
        self.stopwords_file = RefEnv().transformed.format(file) 
        self.stop_words = self.get_set_stopwords()
        
    def get_set_stopwords(self):
        fileName = self.stopwords_file
        stopword_file = open(fileName, 'r')
        list_stop_word = [line.split(',') for line in stopword_file.readlines()][0] # if I understand correctly this enables to get a list of all the stopwords that are in the stopword file spliting with , => why do we use readlines() and what the [0] stands for ? 
        return set(stopwords.words('french')).union(set(stopwords.words('english'))).union(set(list_stop_word)) #here we have 2 sets => at the end can we use the set as a list ? I don't really get how we can use set. Is it only to make sure that we have a unique example for each word ? 


to_remove = [Lemm(file = swrdlist).stop_words for swrdlist in stopwords_list]
all_stop = set()
for element in to_remove:
    all_stop = all_stop.union(element)
    
all_stop = list(all_stop)

In [ ]:
from rake_nltk import Metric, Rake
from DA_IO import Gspreadsheet
import nltk
data = Gspreadsheet('DataStudio - BIG_Challenge1000').sheet_to_df()
print(data.keys())


def rake_description(element, lang):
    dict_lang = {'EN':'english', 'FR': 'french'}
    if lang in ['EN', 'FR']:
        r = Rake(ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO,
                 min_length=1, 
                 max_length=1, 
                 language = dict_lang[lang], 
                 punctuations = [';',"’",'&', "'",',', '.', '3', '2', '1', '(', ')'],
                 stopwords = all_stop
                )
        r.extract_keywords_from_text(element)
        return ' '.join([e.capitalize() for e in r.get_ranked_phrases()])
    else:
        return ''
def run_rake():    
    data['keywords_struct'] = data.apply(lambda x: rake_description(x['prez_struc'], x['Langue']), axis = 1)
    data['keywords_product'] = data.apply(lambda x: rake_description(x['prez_produit_struc'], x['Langue']), axis = 1)
    return data

def update_datastudio():
    Gspreadsheet('DataStudio - BIG_Challenge1000').df_to_sheet(output[col + ['prez','Langue','keywords_struct','keywords_product']])


In [ ]:
output = run_rake()
col = ['Prénom', 'Nom', 'nom_struc', 'Nationalité','cat_sub1_struc', 'cat_sub2_struc', 'cat_sub3_struc', 'cat_struc','cat_autre_struc','catégorie','prez_struc', 'prez_produit_struc']

In [ ]:
for element in output['keywords_product'].values.tolist():
    print(element, '\n')

In [ ]:
update_datastudio()